# Voronoi

> Fill in a module description here

In [ ]:
#| default_exp voronoi 

#| export
## Introduction

Lets see if we can make some terrain

#| export
## Getting Started

In [ ]:
#| export
#data
from collections import namedtuple, defaultdict
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#math
import random
from random import randint
import numpy as np

from fastcore.basics import patch

In [ ]:
#| export
from HexMagic.primitives import  MapCord , HexPosition,  MapSize, MapRect, MapCord, MapPath ,HexGrid, HexRegion, Hex
from HexMagic.primitives import MapRect, HexGrid


In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
from sklearn.cluster import KMeans

In [ ]:
def sampleGrid(hexDim = 2, radius=40, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.centered(hexDim,radius=radius,style=hexStyle)
    #grid.adjustRadius(radius)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        if makeLabels:
            grid.hexes[i].label = str(i) 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    return grid
    

grid = sampleGrid(makeLabels=True)
grid.builder.show()

In [ ]:
def demo_voronoi_growth_simple():
    """Simpler Voronoi growth using HexRegion list"""
    grid = sampleGrid(5, fill="lightgray")
    
    colors = [
        StyleCSS("region0", fill="#ff6b6b", stroke="darkred", stroke_width=0),
        StyleCSS("region1", fill="#4ecdc4", stroke="darkblue", stroke_width=0),
        StyleCSS("region2", fill="#95e1d3", stroke="darkgreen", stroke_width=0),
        StyleCSS("region3", fill="#f38181", stroke="darkorange", stroke_width=0),
    ]
    
    perimeter_style = StyleCSS("perimeter_path", fill="none", stroke="#333", stroke_width=3)
    arrow_style = StyleCSS("growth_arrow", stroke="#333333", stroke_width=2, fill="none")
    boundary_layer = ""
    for style in colors + [perimeter_style, arrow_style]:
        grid.builder.add_style(style)
    
    seeds = [
        grid.nCols * 1 + 1,
        grid.nCols * 1 + (grid.nCols - 2),
        grid.nCols * (grid.nRows - 2) + 1,
        grid.nCols * (grid.nRows - 2) + (grid.nCols - 2),
    ]
    
    regions = [HexRegion({seed}, grid) for seed in seeds]
    
    for i, seed in enumerate(seeds):
        grid.hexes[seed].style = colors[i]
        grid.hexes[seed].label = str(i)
    
    claimed = set(seeds)
    total = grid.nRows * grid.nCols
    arrow_layer = ""
    
    # Grow until everything is claimed
    for round_num in range(30):
        if len(claimed) >= total:
            break
        
        # Collect ALL unclaimed candidates and which regions want them
        candidates = {}  # hex_idx -> [(region_idx, source, dist), ...]
        
        for i, region in enumerate(regions):
            centroid_idx = region.centroid_hex()
            centroid_pos = grid.index_to_hexposition(centroid_idx)
            
            outside = region.outside()
            for candidate in outside.hexes:
                if candidate in claimed:
                    continue
                    
                neighbors = grid.neighborsOf(candidate)
                source = next((n for n in neighbors if n in region.hexes), None)
                
                if source is not None:
                    candidate_pos = grid.index_to_hexposition(candidate)
                    dist = candidate_pos.distance(centroid_pos)
                    
                    if candidate not in candidates:
                        candidates[candidate] = []
                    candidates[candidate].append((i, source, dist))
        
        if not candidates:
            break
        
        # Assign each candidate to closest region (or first if tied)
        for candidate, claimants in candidates.items():
            claimants.sort(key=lambda x: (x[2], x[0]))  # Sort by dist, then region idx for ties
            region_idx, source, dist = claimants[0]
            
            claimed.add(candidate)
            regions[region_idx] = HexRegion(regions[region_idx].hexes | {candidate}, grid)
            grid.hexes[candidate].style = colors[region_idx]
            arrow_layer += grid.arrow(source, candidate, style=arrow_style)
    
        for region in regions:
            paths = region.trace_perimeter(style=perimeter_style)
            boundary_layer += "".join([x.make_windy(iterations=2, offset_factor=0.2).svg() for x in paths])
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    grid.builder.adjust("boundaries", boundary_layer)
    return grid.builder.show()


In [ ]:

demo_voronoi_growth_simple()

In [ ]:
from HexMagic.terrain import Terrain



In [ ]:
#| export


def voronoi_seeds(rows, cols, num_seeds, elimination_rate=0.3, max_offset=3, s=None,debug=False):
    """
    Generate evenly-spaced seeds with randomization for Voronoi diagram.
    
    Args:
        rows: Number of rows in grid
        cols: Number of columns in grid
        num_seeds: Target number of seeds (approximate)
        elimination_rate: Fraction of initial seeds to remove (0-1)
        max_offset: Maximum random offset in each direction
        seed: Random seed for reproducibility
    
    Returns:
        Array of (row, col) coordinates and corresponding 1D indexes
    """
    if s is not None:
        np.random.seed(s)
    

    # Calculate spacing to get approximately num_seeds AFTER elimination
    total_cells = rows * cols
    target_before_elimination = num_seeds / (1 - elimination_rate)
    spacing = int(np.sqrt(total_cells / target_before_elimination))
    spacing = max(1, spacing)

    
    # Create evenly spaced grid
    row_coords = np.arange(spacing//2, rows, spacing)
    col_coords = np.arange(spacing//2, cols, spacing)
    rr, cc = np.meshgrid(row_coords, col_coords)
    seeds = np.column_stack([rr.ravel(), cc.ravel()])
    
    if debug:
        print(f"Initial seeds: {len(seeds)} (spacing={spacing})")
    
    # Elimination step
    keep_mask = np.random.random(len(seeds)) > elimination_rate
    seeds = seeds[keep_mask]
    if debug:
        print(f"After elimination: {len(seeds)}")
    
    # Offset step
    offsets = np.random.uniform(-max_offset, max_offset, size=seeds.shape)
    seeds = seeds + offsets
    
    # Clip to valid range and convert to integers
    seeds[:, 0] = np.clip(seeds[:, 0], 0, rows - 1)
    seeds[:, 1] = np.clip(seeds[:, 1], 0, cols - 1)
    seeds = seeds.astype(int)
    
    # Remove duplicates
    seeds = np.unique(seeds, axis=0)
    if debug:
        print(f"Final unique seeds: {len(seeds)}")
    
    # Convert to 1D indexes
    indexes = seeds[:, 0] * cols + seeds[:, 1]
    
    return seeds, indexes




In [ ]:
# Test with 25x25 grid, targeting 6 seeds
coords, indexes = voronoi_seeds(25, 25, num_seeds=6, s=42)

print("\nSeed coordinates (row, col):")
print(coords)
print("\n1D indexes:")
print(indexes)


Seed coordinates (row, col):
[[ 2 10]
 [ 5  1]
 [12 19]
 [14  5]
 [18  2]
 [18 20]]

1D indexes:
[ 60 126 319 355 452 470]


## Plates

In [ ]:
#| export
class PlateKind(Enum):
    oceanic = 0        # Ocean/sea
    continental = 1    # Lakes/rivers
    island_arc = 2        # Cold, low precipitation
    mixed = 3        # Hot/cold, very low precipitation

In [ ]:
#| export

class Plate:
    def __init__(self,region:HexRegion,symbol="♠️",direction=4):
        self.region = region
        self.symbol = symbol
        self.direction = HexPosition.directions()[direction]
        self.kind = PlateKind.oceanic
        self.oceanDistance = 0
        

    def changeDirection(self,direction):
        self.direction = HexPosition.directions()[direction]


    @property
    def hexes(self):
        return self.region.hexes

    @classmethod
    def colors(cls):
        return {
    PlateKind.oceanic: [
        "#2E5266",  # Deep ocean blue
        "#1A4B68",  # Dark blue-gray
        "#2A5674",  # Steel blue
        "#345D7E",  # Slate blue
        "#3E6888",  # Ocean blue
    ],
    PlateKind.continental: [
        "#8B7355",  # Sandy brown
        "#A08968",  # Warm brown
        "#7A6A53",  # Earth brown
        "#9B8B6F",  # Tan brown
        "#6B5D47",  # Dark earth
    ],
    PlateKind.mixed: [
        "#5C6F7A",  # Blue-gray
        "#6A7B85",  # Slate gray
        "#7A8C95",  # Light slate
        "#4D5E69",  # Dark slate
        "#556673",  # Medium slate
    ],
    PlateKind.island_arc: [
    "#6B8E7F",  # Volcanic green-gray
    "#7A9B8C",  # Sage green
    "#5A7A6B",  # Dark moss
    "#8AA599",  # Light seafoam
    "#4F6B5E",  # Deep forest-gray
]

}
    def encode(self) -> str:
        dir_idx = HexPosition.directions().index(self.direction)
        hexes_str = ','.join(str(h) for h in sorted(self.region.hexes))
        return f"{self.symbol}\t{self.kind.value}\t{dir_idx}\t{hexes_str}"

    @classmethod
    def decode(cls, s: str, hex_grid: HexGrid) -> 'Plate':
        parts = s.split('\t')
        symbol = parts[0]
        kind = PlateKind(int(parts[1]))
        direction = int(parts[2])
        hexes = set(int(h) for h in parts[3].split(','))
        region = HexRegion(hexes, hex_grid=hex_grid)
        plate = cls(region, symbol=symbol, direction=direction)
        plate.kind = kind
        return plate



    @classmethod
    def icon(cls, index:int,plates:['Plate'])->str:
        for plate in plates:
            if index in plate.region.hexes:
                return plate.symbol
        return str(index)

    @classmethod
    def voronoi(cls,grid:HexGrid,amount:int=10,sym=["♣️","♦️","♥️","♠️"])->['Plate']:

        seeds = voronoi_seeds(grid.nRows,grid.nCols,amount)[1]

        plates = []
        claimed = set()
        for i, seed in enumerate(seeds):
            icon = str(i)
            if i < len(sym):
                icon = sym[i]
            reg = HexRegion(set([seed]), hexGrid = grid ) # Set of hex indices
            plate = Plate(reg,symbol=icon)
            claimed.add(seed)
            plates.append(plate)

        total = grid.nRows * grid.nCols 

        while len(claimed) < total:
            for plate in plates:
                reg = plate.region.outside()
                adds =[x for x in reg.hexes if x not in claimed]
                for add in adds:
                    claimed.add(add)
                    plate.hexes.add(add)
        return plates

    @classmethod
    def boundaries(cls: type, plates: list['Plate']) -> set[HexPosition]:
        """Find hexes at edges between adjacent plates."""
        boundary = set()
        plate_map = {}  # position -> plate_id
        
        for i, plate in enumerate(plates):
            for p in plate.positions():
                plate_map[p] = i
        
        for plate_id, plate in enumerate(plates):
            for p in plate.positions():
                for d in range(6):
                    neighbor = p.neighbor(d)
                    neighbor_plate = plate_map.get(neighbor)
                    if neighbor_plate is not None and neighbor_plate != plate_id:
                        boundary.add(p)
                        break
        return boundary

    @classmethod
    def merge_plates(cls, voronoi_plates: List['Plate'], 
                 hex_grid: 'HexGrid',
                 target_count: int = 3,
                 debug: bool = False) -> List['Plate']:
        """Merge small voronoi plates into larger tectonic plates using k-means."""
        
        # Separate by kind
        oceanic = [p for p in voronoi_plates if p.kind == PlateKind.oceanic]
        continental = [p for p in voronoi_plates if p.kind == PlateKind.continental]
        
        if debug:
            print(f"\n=== MERGING PLATES ===")
            print(f"Input: {len(oceanic)} oceanic + {len(continental)} continental = {len(voronoi_plates)} total")
            print(f"Target: {target_count} plates")
        
        # Decide split between oceanic and continental
        total = len(voronoi_plates)
        n_oceanic = max(1, round(target_count * len(oceanic) / total))
        n_continental = max(1, target_count - n_oceanic)
        
        if debug:
            print(f"Splitting: {n_oceanic} oceanic + {n_continental} continental")
        
        merged = []
        
        def cluster_plates(plates, k):
            if len(plates) <= k:
                return [[p] for p in plates]
            
            # Get centroids as (x, y) coordinates
            centroids = []
            for plate in plates:
                center_idx = plate.region.centroid_hex()
                center_hex = hex_grid.hexes[center_idx]
                centroids.append((center_hex.center.x, center_hex.center.y))
            
            centroids = np.array(centroids)
            
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(centroids)
            
            clusters = defaultdict(list)
            for plate, label in zip(plates, labels):
                clusters[label].append(plate)
            
            return list(clusters.values())
        
        # Cluster and merge oceanic plates
        if oceanic:
            oceanic_clusters = cluster_plates(oceanic, n_oceanic)
            for cluster in oceanic_clusters:
                all_hexes = set()
                for plate in cluster:
                    all_hexes.update(plate.hexes)
                
                region = HexRegion(hexes=all_hexes, hexGrid=hex_grid)
                merged_plate = Plate(region, symbol="🌊")
                merged_plate.kind = PlateKind.oceanic
                merged.append(merged_plate)
        
        # Cluster and merge continental plates
        if continental:
            continental_clusters = cluster_plates(continental, n_continental)
            for cluster in continental_clusters:
                all_hexes = set()
                for plate in cluster:
                    all_hexes.update(plate.hexes)
                
                region = HexRegion(hexes=all_hexes, hexGrid=hex_grid)
                merged_plate = Plate(region, symbol="🏔️")
                merged_plate.kind = PlateKind.continental
                merged.append(merged_plate)
        
        if debug:
            print(f"\nOutput: {len(merged)} merged plates")
            for i, plate in enumerate(merged):
                print(f"  Plate {i} ({plate.kind.name}): {len(plate.hexes)} hexes")
        
        return merged

In [ ]:
#| export
@patch
def positions(self: Plate) -> set[HexPosition]:
    """Get all hex positions in this plate."""
    return self.region.to_positions()

@patch
def centroid_position(self: Plate) -> HexPosition:
    """Get plate centroid as HexPosition."""
    return self.region.centroid_position()

@patch
def subdivide(self: Plate, depth: int = 2, min_size: int = 10, 
              seeds_per_level: int = 3) -> list['Plate']:
    """Recursively subdivide this plate using Voronoi."""
    positions = self.positions()
    
    if depth <= 0 or len(positions) < min_size:
        return [self]
    
    pos_list = list(positions)
    n_seeds = min(seeds_per_level, len(pos_list))
    seeds = random.sample(pos_list, n_seeds)
    
    # Assign each position to nearest seed
    regions: dict[int, set[HexPosition]] = {}
    for p in positions:
        closest = min(range(len(seeds)), key=lambda i: p.distance(seeds[i]))
        regions[closest] = regions.get(closest, set())
        regions[closest].add(p)
    
    # Convert back to index-based HexRegions and create sub-plates
    sub_plates = []
    for region_positions in regions.values():
        indices = {self.region.hexGrid.hexposition_to_index(p) for p in region_positions}
        indices = {i for i in indices if i >= 0}  # Filter invalid
        if indices:
            sub_region = HexRegion(indices, hexGrid=self.region.hexGrid)
            sub_plate = Plate(sub_region, symbol=self.symbol)
            sub_plate.kind = self.kind
            
            # Recurse
            if depth > 1 and len(indices) >= min_size:
                sub_plates.extend(sub_plate.subdivide(depth - 1, min_size, seeds_per_level))
            else:
                sub_plates.append(sub_plate)
    
    return sub_plates




In [ ]:
#| export
@patch
def plateOverlay(grid:HexGrid,plates:['Plate'],palette="hls",f=None,smooth=False):
    
    styles = StyleCSS.seaborn(palette, len(plates))

    for i, plate in enumerate(plates):
        plateStyle = styles[i]
        grid.builder.add_style(plateStyle)
        for k in plate.hexes:
            grid.hexes[k].style = plateStyle

    return grid.styleLayer(f=f,smooth=smooth)

In [ ]:
def demoVornoi(rings=5,points=10,radius=20):
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid,points)
    grid.builder.adjust("Plates",grid.plateOverlay(plates))
    return grid

In [ ]:
vern = demoVornoi(rings=15,points=80,radius=10)
vern.builder.show()

In [ ]:
#| export
def mark_oceanic_plates(plates, ocean_fraction=0.4, seed=None):
    """
    Randomly mark some plates as oceanic.
    
    Args:
        plates: List of Plate objects
        ocean_fraction: Fraction of plates to mark as oceanic (0-1)
        seed: Random seed for reproducibility
    
    Returns:
        Modified plates list
    """
    if seed is not None:
        np.random.seed(seed)
    
    num_ocean = int(len(plates) * ocean_fraction)
    ocean_indices = np.random.choice(len(plates), size=num_ocean, replace=False)
    
    for i in ocean_indices:
        plates[i].kind = PlateKind.oceanic
    
    # Mark the rest as continental
    for i, plate in enumerate(plates):
        if i not in ocean_indices:
            plate.kind = PlateKind.continental
    
    return plates




In [ ]:
#| export
def mark_oceanic_by_sides(plates, grid, factor=1.5, oceanic_sides=['N', 'E', 'S', 'W']):
    """
    Mark plates as oceanic if they touch specified map edges.
    
    Args:
        plates: List of Plate objects
        grid: HexGrid
        bounds: MapRect with min_x, max_x, min_y, max_y
        oceanic_sides: List of sides - 'N', 'E', 'S', 'W'
                      e.g. ['N','E','S'] for a peninsula open to the west
    """
    # Build a set of edge hex indices for each side
    edge_hexes = set()
    margin = grid.radius * factor  # tolerance for edge detection
    bounds = grid.bounds
    max_y = bounds.maxY() - margin
    min_y = bounds.minY() + margin
    max_x = bounds.maxX() - margin
    min_x = bounds.minX() + margin
  
    for i, hx in enumerate(grid.hexes):
        
        x = hx.center.x
        y = hx.center.y
        
        
        if 'N' in oceanic_sides and y <= min_y :
            edge_hexes.add(i)
        if 'S' in oceanic_sides and y >= max_y:
            edge_hexes.add(i)
        if 'E' in oceanic_sides and x >= max_x:
            edge_hexes.add(i)
        if 'W' in oceanic_sides and x <= min_x:
            edge_hexes.add(i)
    
    # Mark plates that intersect with edge hexes as oceanic
    for plate in plates:
        plate_hexes = set(plate.hexes)
        if plate_hexes & edge_hexes:  # intersection
            plate.kind = PlateKind.oceanic

    
    return plates


In [ ]:
#| export
@patch
def calculate_distances_from_ocean(grid:HexGrid,plates:[Plate]):
    """
    Calculate distance from each plate to nearest oceanic plate centroid.
    """
  
    # Get oceanic plate centroids
    ocean_centroids = []
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            centroid_idx = plate.region.centroid_hex()
            centroid_pos = grid.index_to_hexposition(centroid_idx)
            ocean_centroids.append(centroid_pos)
    
    if not ocean_centroids:
        print("Warning: No oceanic plates found!")
        return 

    for plate in plates:
        if plate.kind != PlateKind.oceanic:
            centroid_idx = plate.region.centroid_hex()
            hex_pos = grid.index_to_hexposition(centroid_idx)
            plate.oceanDistance = min(hex_pos.distance(ocean_pos) for ocean_pos in ocean_centroids)
   


@patch
def adjust_plate_heights(terrain: 'Terrain', plates: list[Plate],
                        reference_hexes: set[HexPosition] = None,
                        height_func=None) -> None:
    """Adjust terrain height based on distance from plate centroids to reference hexes."""
    if reference_hexes is None:
        reference_hexes = Plate.boundaries(plates)
    
    if height_func is None:
        height_func = lambda base, dist: base + max(0, 5 - dist) * 0.5
    
    grid = plates[0].region.hexGrid
    
    for plate in plates:
        centroid = plate.centroid_position()
        min_dist = min((centroid.distance(ref) for ref in reference_hexes), default=float('inf'))
        
        for idx in plate.hexes:
            if idx < len(terrain.elevations):
                terrain.elevations[idx] = height_func(terrain.elevations[idx], min_dist)

@patch
def adjust_by_distance(terrain: 'Terrain', plate: Plate,
                       from_pos: HexPosition = None,
                       height_func=None) -> None:
    """
    Adjust terrain height for hexes in a plate based on distance from a reference point.
    
    Args:
        terrain: Terrain object with elevations
        plate: Single plate to adjust
        from_pos: Reference position (defaults to plate centroid)
        height_func: fn(base_height, distance) -> new_height
                    Default creates a peak at center that falls off with distance
    """
    grid = plate.region.hexGrid
    
    if from_pos is None:
        from_pos = plate.centroid_position()
    
    if height_func is None:
        # Default: peak at center, falls off linearly
        height_func = lambda base, dist: base + max(0, 8 - dist * 1.5)
    
    for idx in plate.hexes:
        if idx < len(terrain.elevations):
            hex_pos = grid.index_to_hexposition(idx)
            dist = hex_pos.distance(from_pos)
            terrain.elevations[idx] = height_func(terrain.elevations[idx], dist)


lets make a similar function but adjusts a the hexes of a plate by the ditance to an arbitary hex. if no position is passed in, pick the centroid

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's a function that adjusts hex heights within a plate based on distance from a reference point:

```python
@patch
def adjust_by_distance(terrain: 'Terrain', plate: Plate,
                       from_pos: HexPosition = None,
                       height_func=None) -> None:
    """
    Adjust terrain height for hexes in a plate based on distance from a reference point.
    
    Args:
        terrain: Terrain object with elevations
        plate: Single plate to adjust
        from_pos: Reference position (defaults to plate centroid)
        height_func: fn(base_height, distance) -> new_height
                    Default creates a peak at center that falls off with distance
    """
    grid = plate.region.hexGrid
    
    if from_pos is None:
        from_pos = plate.centroid_position()
    
    if height_func is None:
        # Default: peak at center, falls off linearly
        height_func = lambda base, dist: base + max(0, 8 - dist * 1.5)
    
    for idx in plate.hexes:
        if idx < len(terrain.elevations):
            hex_pos = grid.index_to_hexposition(idx)
            dist = hex_pos.distance(from_pos)
            terrain.elevations[idx] = height_func(terrain.elevations[idx], dist)
```

This gives you flexibility - you can pass a custom `height_func` to create different shapes:

```python
# Mountain peak (sharp falloff)
mountain_func = lambda base, dist: base + max(0, 10 - dist * 2)

# Gentle dome
dome_func = lambda base, dist: base + max(0, 5 - dist * 0.5)

# Crater (low center, high rim)
crater_func = lambda base, dist: base + (3 if 2 < dist < 4 else 0)
```

In [ ]:
def demoOceanicDistance(rings=5, points=10, radius=20, ocean_fraction=0.1,factor=4, oceanic_sides=[]):
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid, points)
    
    # Mark some as oceanic
    plates = mark_oceanic_plates(plates, ocean_fraction=ocean_fraction, seed=42)
    if len(oceanic_sides) > 0:
        mark_oceanic_by_sides(plates, grid, factor=factor, oceanic_sides=oceanic_sides)

    
    # Calculate distances
    grid.calculate_distances_from_ocean(plates)

    levels = max([x.oceanDistance for x in plates])
    styles = StyleCSS.seaborn("terrain", levels + 1)
 
    
    # Color plates by type
    for plate in plates:
        plateStyle = styles[plate.oceanDistance]
        grid.builder.add_style(plateStyle)
        for k in plate.hexes:
            grid.hexes[k].style = plateStyle


    grid.builder.adjust("regions", grid.styleLayer())
    #grid.update()
    
    # Print some stats
    print(f"Total plates: {len(plates)}")
    print(f"Oceanic plates: {sum(1 for p in plates if p.kind == PlateKind.oceanic)}")

    
    return grid , plates


In [ ]:
dOD , dPlates = demoOceanicDistance(rings=25,points=300,radius=5, ocean_fraction=0.25,factor=6, oceanic_sides=['N'])
dOD.builder.show()

Total plates: 398
Oceanic plates: 132


In [ ]:
dOD , dPlates = demoOceanicDistance(rings=25,points=40,radius=15, ocean_fraction=0.25,factor=6, oceanic_sides=['N',"W"])
dOD.builder.show()

Total plates: 42
Oceanic plates: 19


In [ ]:
dOD , dPlates = demoOceanicDistance(rings=25,points=25,radius=10, ocean_fraction=0.1,factor=3, oceanic_sides=['N','S','E','W'])
dOD.builder.show()

Total plates: 26
Oceanic plates: 16


In [ ]:
def demoMerge(grid,plates):
    #print(grid)
    #print(plates)
    
    # Mark some as oceanic
    plates = Plate.merge_plates(plates,grid, target_count=4)
    
    # Calculate distances
    grid.calculate_distances_from_ocean(plates)

    levels = max([x.oceanDistance for x in plates])
    styles = StyleCSS.seaborn("terrain", levels + 1)
 
    
    # Color plates by type
    for plate in plates:
        plateStyle = styles[plate.oceanDistance]
        grid.builder.add_style(plateStyle)
        for k in plate.hexes:
            grid.hexes[k].style = plateStyle


    grid.builder.adjust("regions", grid.styleLayer())
    #grid.update()
    
    # Print some stats
    print(f"Total plates: {len(plates)}")
    print(f"Oceanic plates: {sum(1 for p in plates if p.kind == PlateKind.oceanic)}")

    
    return grid, plates

In [ ]:
sampleGrid, bigPlates = demoMerge(dOD,dPlates)
sampleGrid.builder.show()

Total plates: 4
Oceanic plates: 2


In [ ]:
??calculate_distances_from_ocean

fix?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The error is happening because `calculate_distances_from_ocean` is returning `None` instead of being a callable function. Looking at your code, it seems this function might not be defined or imported correctly.

Let me help you fix this by creating the missing function:

```python
def calculate_distances_from_ocean(plates, grid):
    """Calculate distance from ocean for each plate using BFS."""
    from collections import deque
    
    # First, set all plates to unvisited
    for plate in plates:
        plate.oceanDistance = None
    
    # Initialize queue with oceanic plates
    queue = deque()
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            plate.oceanDistance = 0
            queue.append(plate)
    
    # BFS to calculate distances
    while queue:
        current = queue.popleft()
        current_dist = current.oceanDistance
        
        # Find neighboring plates
        neighbors = set()
        for hex_key in current.hexes:
            hex_obj = grid.hexes[hex_key]
            for neighbor_key in grid.neighbors(hex_obj):
                neighbor_hex = grid.hexes[neighbor_key]
                if hasattr(neighbor_hex, 'plate') and neighbor_hex.plate:
                    neighbors.add(neighbor_hex.plate)
        
        # Update distances for unvisited neighbors
        for neighbor in neighbors:
            if neighbor.oceanDistance is None:
                neighbor.oceanDistance = current_dist + 1
                queue.append(neighbor)
```

Now run your demo again:

```python
sampleGrid, bigPlates = demoMerge(dOD, dPlates)
sampleGrid.builder.show()
```

This function uses breadth-first search (BFS) to calculate the distance of each plate from the nearest oceanic plate, which creates the nice gradient effect you're seeing in your terrain visualization.

In [ ]:
smallPlates = []
for plate in bigPlates:
    smallPlates.extend(plate.subdivide(depth = 3, min_size = 3))

sampleGrid.calculate_distances_from_ocean(smallPlates)

levels = max([x.oceanDistance for x in smallPlates])
styles = StyleCSS.seaborn("terrain", levels + 1)


# Color plates by type
for i, plate in enumerate(smallPlates):
    
    plateStyle = styles[plate.oceanDistance]
    sampleGrid.builder.add_style(plateStyle)
    #print(f"{i} has style {plate.oceanDistance}  {plateStyle.name} for hexes {len(plate.hexes)}")
    for k in plate.hexes:
        sampleGrid.hexes[k].style = plateStyle

sampleGrid.builder.layers = []
sampleGrid.builder.adjust("regions", sampleGrid.styleLayer())
#grid.update()

# Print some stats
print(f"Total plates: {len(smallPlates)}")
print(f"Oceanic plates: {sum(1 for p in smallPlates if p.kind == PlateKind.oceanic)}")
sampleGrid.builder.show()



Total plates: 108
Oceanic plates: 54


think of demos that would use adjust_plate_heights. right now we are creating moutains in the middle. maybe we want ridges

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question! The current approach creates central mountains, but real terrain forms at plate *boundaries*. Here are some demo ideas:

**1. Collision Ridges** — Mountains along boundaries between continental plates
```python
def ridge_height_func(base, dist_to_boundary):
    # Height peaks AT the boundary, falls off with distance
    return base + max(0, 3 - dist_to_boundary) * 2
```

**2. Subduction Zones** — Asymmetric: trench on ocean side, mountains on continental side
```python
def subduction_demo(plates, grid):
    for hex_idx in boundary_hexes:
        ocean_neighbor = is_adjacent_to_oceanic(hex_idx)
        continental_neighbor = is_adjacent_to_continental(hex_idx)
        if ocean_neighbor and continental_neighbor:
            # Create trench on ocean side, uplift on continental
            ...
```

**3. Mid-Ocean Ridges** — Elevated ridges where oceanic plates meet
```python
def ocean_ridge_func(base, dist, plate_kind):
    if plate_kind == PlateKind.oceanic:
        return base - 2 + max(0, 2 - dist)  # Underwater ridge
    return base
```

**4. Rift Valleys** — Depressions where plates diverge (based on direction vectors)
```python
def rift_valley(plate1, plate2, boundary):
    # If plates moving apart, lower the boundary
    if plates_diverging(plate1, plate2):
        return base - 3
```

**5. Distance-from-ocean elevation** — What you have now but inverted for coastlines
```python
def coastal_slope(base, ocean_dist):
    return base + min(ocean_dist * 0.5, 5)  # Gradual rise inland
```

Which approach interests you most? The collision ridge one would be the most natural next step given your current `Plate.boundaries()` method.

So when 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
def demo_boundary_ridges(rings=10, points=15, radius=10, oceanic_sides=['N', 'E', 'S', 'W']):
    """Demo showing mountain ridges at plate boundaries."""
    from HexMagic.terrain import Terrain
    
    grid = sampleGrid(rings, radius=radius, fill="lightgray")
    plates = Plate.voronoi(grid, points)
    
    # Mark edge plates as oceanic
    mark_oceanic_by_sides(plates, grid, factor=2, oceanic_sides=oceanic_sides)
    
    # Subdivide for finer detail
    small_plates = []
    for plate in plates:
        small_plates.extend(plate.subdivide(depth=3, min_size=3))
    
    # Get boundaries between plates
    boundaries = Plate.boundaries(small_plates)
    
    # Create terrain with base elevations
    terrain = Terrain(grid)
    terrain.elevations = [0] * len(grid.hexes)
    
    # For each hex, height based on distance to nearest boundary
    for i, hx in enumerate(grid.hexes):
        pos = grid.index_to_hexposition(i)
        
        # Find which plate this hex belongs to
        plate_kind = PlateKind.oceanic
        for plate in small_plates:
            if i in plate.hexes:
                plate_kind = plate.kind
                break
        
        # Distance to nearest boundary
        if boundaries:
            dist_to_boundary = min(pos.distance(b) for b in boundaries)
        else:
            dist_to_boundary = 10
        
        # Ridge formula: peak near boundaries
        if plate_kind == PlateKind.continental:
            # Continental: ridges at boundaries, plateau inland
            ridge_height = max(0, 8 - dist_to_boundary * 1.5)
            base_height = 2
            terrain.elevations[i] = base_height + ridge_height
        else:
            # Oceanic: trenches near boundaries, flat elsewhere
            trench_depth = max(0, 3 - dist_to_boundary * 0.8)
            terrain.elevations[i] = -trench_depth
    
    # Normalize to terrain levels
    min_elev = min(terrain.elevations)
    max_elev = max(terrain.elevations)
    elev_range = max_elev - min_elev if max_elev != min_elev else 1
    
    num_levels = 12
    styles = StyleCSS.seaborn("terrain", num_levels)
    
    for i, elev in enumerate(terrain.elevations):
        level = int((elev - min_elev) / elev_range * (num_levels - 1))
        level = max(0, min(num_levels - 1, level))
        style = styles[level]
        grid.builder.add_style(style)
        grid.hexes[i].style = style
    
    grid.builder.adjust("terrain", grid.styleLayer(smooth=True))
    
    print(f"Plates: {len(small_plates)}, Boundary hexes: {len(boundaries)}")
    print(f"Elevation range: {min_elev:.1f} to {max_elev:.1f}")
    
    return grid, small_plates, terrain

grid, plates, terrain = demo_boundary_ridges()
grid.builder.show()
```

This creates mountain ridges along plate boundaries rather than at centroids. Continental plates get ridges where they meet (like collision zones), while oceanic plates get slight trenches near boundaries. Try adjusting:
- `oceanic_sides` - e.g. `['W']` for a west coast continent
- The ridge formula coefficients for sharper/broader ridges
- `points` for more/fewer plates

## Terrain

In [ ]:
#| export
from HexMagic.terrain import Terrain

In [ ]:
@patch
def adjust_plate_heights(terrain: 'Terrain', plates: list[Plate],
                        reference_hexes: set[HexPosition] = None,
                        height_func=None) -> None:
    """Adjust terrain height based on distance from plate centroids to reference hexes.
        This could be used as a replacement on ocean distances
    """
    if reference_hexes is None:
        reference_hexes = Plate.boundaries(plates)
    
    if height_func is None:
        height_func = lambda base, dist: base + max(0, 5 - dist) * 0.5
    
    grid = plates[0].region.hexGrid
    
    for plate in plates:
        centroid = plate.centroid_position()
        min_dist = min((centroid.distance(ref) for ref in reference_hexes), default=float('inf'))
        
        for idx in plate.hexes:
            if idx < len(terrain.elevations):
                terrain.elevations[idx] = height_func(terrain.elevations[idx], min_dist)

In [ ]:
#| export


@patch
def adjust_by_distance(terrain: Terrain, plate: Plate,
                       from_pos: HexPosition = None,
                       height_func=None) -> None:
    """
    Adjust terrain height for hexes in a plate based on distance from a reference point.
    
    Args:
        terrain: Terrain object with elevations
        plate: Single plate to adjust
        from_pos: Reference position (defaults to plate centroid)
        height_func: fn(base_height, distance) -> new_height
                    Default creates a peak at center that falls off with distance
    """
    grid = plate.region.hexGrid
    
    if from_pos is None:
        from_pos = plate.centroid_position()
    
    if height_func is None:
        # Default: peak at center, falls off linearly
        height_func = lambda base, dist: base + max(0, 8 - dist * 1.5)
    
    for idx in plate.hexes:
        if idx < len(terrain.elevations):
            hex_pos = grid.index_to_hexposition(idx)
            dist = hex_pos.distance(from_pos)
            terrain.elevations[idx] = height_func(terrain.elevations[idx], dist)

#| export
This is a prior reference function

```

def generate_plate_terrain(bounds, radius=20, num_plates=10,  slope=20,variation=40, ocean_fraction=0.4, 
factor=1.5, oceanic_sides=[],seed=None):
    """
    Generate terrain based on tectonic plates using Voronoi regions.
    
    Args:
        bounds: MapRect defining the terrain bounds
        radius: Hex radius
        num_plates: Number of tectonic plates
        ocean_fraction: Fraction of plates that are oceanic
        seed: Random seed for reproducibility
    
    Returns:
        Terrain object with plate-based elevations
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Generate Voronoi plates
    plates = Plate.voronoi(grid, num_plates)
    
    # Mark some as oceanic
    plates = mark_oceanic_plates(plates, ocean_fraction=ocean_fraction, seed=seed)

    if len(oceanic_sides) > 0:
        mark_oceanic_by_sides(plates, grid, factor=factor, oceanic_sides=oceanic_sides)

    
    # Calculate distances from ocean
    calculate_distances_from_ocean(plates, grid)
    
    # Generate elevations based on plate properties
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            # Ocean plates are low elevation (negative)
            base_elevation = -100
            variation = 30
        else:
            # Continental plates elevation based on distance from ocean
            # Farther from ocean = higher elevation
            base_elevation = 50 + (plate.oceanDistance * slope)
            
        
        # Add random variation to each hex in the plate
        for hex_idx in plate.hexes:
            elevation = base_elevation + np.random.uniform(-variation, variation)
            elevations[hex_idx] = elevation
    
    # Smooth the elevations at plate boundaries
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            # Average with neighbors for smoothing
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    
    terrain.elevations = smoothed
    terrain.colorMap()
    #if final_plates is not None:
    #    plates = _merge_plates(plates,grid, target_count=final_plates)
    
    return terrain, plates 



Terrain.fromSeeds = generate_plate_terrain
```

I was thinking about refactoring generate_plate_terrain and have a couple of thoughts
1. We definitely should still pass in ocean_sides, ocean_fraction, and factor for how much we adjust the edges
2. having an age - young, middle, old to determine slope and peak heights.
3. does a mountain type make sense. Not it terms of soil, but do we want ocean distance ones, ridges, rolling or others.
4. doing an intital pass with a few seeds works to create great coastlines and I think subdividing afterwards makes a great deal of sense.

anything I am missing? I do like an intial distance from ocean, but afterwards we could be more creative.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Excellent refactoring ideas! Here's what I think:

**Your points are solid:**

1. ✅ **Ocean parameters** - Definitely keep these, they're fundamental to realistic geography
2. ✅ **Age parameter** - Brilliant! Young = sharp peaks/deep trenches, Old = eroded/gentle. Could affect:
   - Elevation range (young: ±200, old: ±50)
   - Smoothing passes (young: 1-2, old: 5-10)
   - Variation within plates
3. ✅ **Mountain type** - Yes! I'd call it `terrain_style` or `formation_type`:
   - `'ocean_distance'` - current approach
   - `'ridge'` - mountains at boundaries
   - `'volcanic'` - peaks at plate centroids
   - `'rift'` - valleys at divergent boundaries
   - `'rolling'` - low variation, gentle hills
4. ✅ **Two-phase approach** - Smart! Coarse plates → coastlines → subdivide → detail

**What you might be missing:**

5. **Erosion/weathering** - Related to age, but could be separate:
   - River valleys cutting through (flow from high→low)
   - Coastal erosion smoothing
   
6. **Plate movement direction** - You already have `plate.direction`:
   - Converging plates → ridges
   - Diverging plates → rifts/valleys
   - Transform faults → linear features
   
7. **Climate influence** - Distance from ocean affects:
   - Weathering rate (coastal = more erosion)
   - Could modify the smoothing differently by region

**Suggested signature:**
```python
def generate_plate_terrain(
    bounds, 
    radius=20,
    # Plate generation
    num_plates=10,
    final_subdivisions=3,
    # Ocean configuration  
    ocean_fraction=0.4,
    oceanic_sides=[],
    edge_factor=1.5,
    # Terrain character
    age='middle',  # 'young', 'middle', 'old'
    formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rift', 'rolling'
    # Fine-tuning
    elevation_scale=1.0,
    variation=40,
    seed=None
):
```

Does this align with your vision?

This aligns better. I am going to do some river work later, but I need this as a start. if there is something simple we could do with plates in terms of Plate movement direction. right now these plates are more geometric (which is fine from a drawing perspective) if there is a way we combine them to get good ridge boundaries that would be great. But if that is only going to be fine tuning it is ok to skip

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I think **skip plate movement direction for now**. Here's why:

**The geometric approach is actually good enough because:**
- Your subdivide method already creates irregular, natural-looking boundaries
- Real ridge formation depends on *which* plates are colliding (continental vs oceanic), not just direction
- The `formation_type='ridge'` approach using `Plate.boundaries()` will give you mountains at plate edges regardless of movement

**What WILL matter for ridges:**
- **Plate type adjacency**: Continental-Continental = high ridges (Himalayas), Oceanic-Continental = asymmetric (Andes), Oceanic-Oceanic = island arcs
- **Distance to boundary**: Already have this with your boundaries set

**Simple enhancement you could add NOW:**
```python
def get_boundary_type(plate1, plate2):
    """Returns 'continental', 'oceanic', or 'mixed' boundary."""
    if plate1.kind == plate2.kind == PlateKind.continental:
        return 'continental'  # Highest mountains
    elif plate1.kind == plate2.kind == PlateKind.oceanic:
        return 'oceanic'  # Trenches/island arcs
    else:
        return 'mixed'  # Subduction zones
```

Then in ridge formation, continental boundaries get 2x height multiplier, mixed gets 1.5x, oceanic gets trenches.

**My recommendation:** 
1. Build the refactored `generate_plate_terrain` with `formation_type` options
2. Get ridge boundaries working with simple distance-based heights
3. Add the boundary_type enhancement above if it's < 20 lines
4. Save directional vectors for later when you do erosion/rivers (they'll matter more there)

Sound good?

Lets start rebuilding the generate_plate_terrain

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
def generate_plate_terrain(
    bounds, 
    radius=20,
    # Plate generation
    num_plates=10,
    subdivisions=3,
    min_subplate_size=3,
    # Ocean configuration  
    ocean_fraction=0.4,
    oceanic_sides=[],
    edge_factor=1.5,
    # Terrain character
    age='middle',  # 'young', 'middle', 'old'
    formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rolling'
    # Fine-tuning
    elevation_scale=1.0,
    seed=None
):
    """
    Generate terrain based on tectonic plates.
    
    Args:
        bounds: MapRect defining terrain bounds
        radius: Hex radius
        num_plates: Initial number of large plates (for coastlines)
        subdivisions: Depth of plate subdivision for detail
        min_subplate_size: Minimum hex count per subplate
        ocean_fraction: Fraction of initial plates marked oceanic
        oceanic_sides: List of sides ['N','E','S','W'] that are ocean
        edge_factor: How far from edge to mark as oceanic
        age: 'young' (sharp), 'middle', 'old' (eroded)
        formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
        elevation_scale: Multiplier for all elevations
        seed: Random seed
    
    Returns:
        (Terrain, plates) tuple
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Age presets
    age_params = {
        'young':  {'slope': 30, 'variation': 50, 'smoothing': 1, 'peak_height': 200},
        'middle': {'slope': 20, 'variation': 40, 'smoothing': 3, 'peak_height': 150},
        'old':    {'slope': 10, 'variation': 20, 'smoothing': 6, 'peak_height': 80},
    }
    params = age_params.get(age, age_params['middle'])
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Phase 1: Generate coarse plates for coastlines
    coarse_plates = Plate.voronoi(grid, num_plates)
    
    # Mark oceanic plates
    coarse_plates = mark_oceanic_plates(coarse_plates, ocean_fraction=ocean_fraction, seed=seed)
    if oceanic_sides:
        mark_oceanic_by_sides(coarse_plates, grid, factor=edge_factor, oceanic_sides=oceanic_sides)
    
    # Phase 2: Subdivide for detail (preserving plate kind)
    plates = []
    for plate in coarse_plates:
        sub = plate.subdivide(depth=subdivisions, min_size=min_subplate_size)
        plates.extend(sub)
    
    # Calculate ocean distances
    grid.calculate_distances_from_ocean(plates)
    
    # Get boundaries for ridge-based terrain
    boundaries = Plate.boundaries(plates)
    
    # Initialize elevations
    elevations = np.zeros(len(grid.hexes))
    
    # Generate elevations based on formation type
    if formation_type == 'ocean_distance':
        elevations = _elevation_ocean_distance(grid, plates, params, elevation_scale)
    elif formation_type == 'ridge':
        elevations = _elevation_ridge(grid, plates, boundaries, params, elevation_scale)
    elif formation_type == 'volcanic':
        elevations = _elevation_volcanic(grid, plates, params, elevation_scale)
    elif formation_type == 'rolling':
        elevations = _elevation_rolling(grid, plates, params, elevation_scale)
    else:
        raise ValueError(f"Unknown formation_type: {formation_type}")
    
    # Apply smoothing based on age
    for _ in range(params['smoothing']):
        elevations = _smooth_elevations(grid, elevations)
    
    terrain.elevations = elevations
    terrain.colorMap()
    
    return terrain, plates


def _elevation_ocean_distance(grid, plates, params, scale):
    """Original approach: elevation based on distance from ocean."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -100 * scale
            var = params['variation'] * 0.5
        else:
            base = (50 + plate.oceanDistance * params['slope']) * scale
            var = params['variation']
        
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_ridge(grid, plates, boundaries, params, scale):
    """Mountains form at plate boundaries."""
    elevations = np.zeros(len(grid.hexes))
    
    # Build plate lookup
    plate_map = {}
    for plate in plates:
        for idx in plate.hexes:
            plate_map[idx] = plate
    
    for i, hx in enumerate(grid.hexes):
        plate = plate_map.get(i)
        if plate is None:
            continue
        
        pos = grid.index_to_hexposition(i)
        
        # Distance to nearest boundary
        if boundaries:
            dist_to_boundary = min(pos.distance(b) for b in boundaries)
        else:
            dist_to_boundary = 10
        
        if plate.kind == PlateKind.oceanic:
            # Ocean: slight trench near boundaries
            base = -80 * scale
            boundary_effect = -max(0, 2 - dist_to_boundary) * 10 * scale
        else:
            # Continental: ridges at boundaries
            base = 30 * scale
            ridge_height = max(0, 4 - dist_to_boundary) * params['peak_height'] / 4 * scale
            boundary_effect = ridge_height
        
        var = params['variation'] * 0.5
        elevations[i] = base + boundary_effect + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_volcanic(grid, plates, params, scale):
    """Peaks at plate centroids (volcanic islands, mountain cores)."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        centroid = plate.centroid_position()
        
        if plate.kind == PlateKind.oceanic:
            base = -60 * scale
            peak = params['peak_height'] * 0.3 * scale  # Underwater volcanoes
        else:
            base = 40 * scale
            peak = params['peak_height'] * scale
        
        for idx in plate.hexes:
            pos = grid.index_to_hexposition(idx)
            dist = pos.distance(centroid)
            
            # Peak at center, falls off
            height = max(0, peak - dist * params['slope'])
            var = params['variation'] * 0.3
            elevations[idx] = base + height + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_rolling(grid, plates, params, scale):
    """Gentle rolling terrain with minimal features."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -30 * scale
        else:
            base = 20 + plate.oceanDistance * params['slope'] * 0.3
            base *= scale
        
        var = params['variation'] * 0.7
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _smooth_elevations(grid, elevations):
    """Single smoothing pass averaging with neighbors."""
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    return smoothed


Terrain.fromSeeds = generate_plate_terrain
```

Now test it:

```python
# Test different formation types
bounds = MapRect(0, 0, 400, 400)

# Ocean distance (original style)
t1, p1 = Terrain.fromSeeds(bounds, radius=8, num_plates=6, oceanic_sides=['N','E'], 
                            formation_type='ocean_distance', age='middle', seed=42)
t1.hexGrid.builder.show()
```

```python
# Ridge mountains
t2, p2 = Terrain.fromSeeds(bounds, radius=8, num_plates=6, oceanic_sides=['N','E'],
                            formation_type='ridge', age='young', seed=42)
t2.hexGrid.builder.show()
```

```python
# Volcanic peaks
t3, p3 = Terrain.fromSeeds(bounds, radius=8, num_plates=6, oceanic_sides=['N','E'],
                            formation_type='volcanic', age='old', seed=42)
t3.hexGrid.builder.show()
```

The structure is:
- **Phase 1**: Coarse plates define coastlines
- **Phase 2**: Subdivision adds detail
- **Formation functions**: Each returns elevations array
- **Age**: Controls slope, variation, smoothing, peak heights

In [ ]:
#| export
def generate_plate_terrain(
    bounds, 
    radius=20,
    # Plate generation
    num_plates=10,
    subdivisions=3,
    min_subplate_size=3,
    # Ocean configuration  
    ocean_fraction=0.4,
    oceanic_sides=[],
    edge_factor=1.5,
    # Terrain character
    age='middle',  # 'young', 'middle', 'old'
    formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rolling'
    # Fine-tuning
    elevation_scale=1.0,
    seed=None
):
    """
    Generate terrain based on tectonic plates.
    
    Args:
        bounds: MapRect defining terrain bounds
        radius: Hex radius
        num_plates: Initial number of large plates (for coastlines)
        subdivisions: Depth of plate subdivision for detail
        min_subplate_size: Minimum hex count per subplate
        ocean_fraction: Fraction of initial plates marked oceanic
        oceanic_sides: List of sides ['N','E','S','W'] that are ocean
        edge_factor: How far from edge to mark as oceanic
        age: 'young' (sharp), 'middle', 'old' (eroded)
        formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
        elevation_scale: Multiplier for all elevations
        seed: Random seed
    
    Returns:
        (Terrain, plates) tuple
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Age presets
    age_params = {
        'young':  {'slope': 30, 'variation': 50, 'smoothing': 1, 'peak_height': 200},
        'middle': {'slope': 20, 'variation': 40, 'smoothing': 3, 'peak_height': 150},
        'old':    {'slope': 10, 'variation': 20, 'smoothing': 6, 'peak_height': 80},
    }
    params = age_params.get(age, age_params['middle'])
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Phase 1: Generate coarse plates for coastlines
    coarse_plates = Plate.voronoi(grid, num_plates)
    
    # Mark oceanic plates
    coarse_plates = mark_oceanic_plates(coarse_plates, ocean_fraction=ocean_fraction, seed=seed)
    if oceanic_sides:
        mark_oceanic_by_sides(coarse_plates, grid, factor=edge_factor, oceanic_sides=oceanic_sides)
    
    # Phase 2: Subdivide for detail (preserving plate kind)
    plates = []
    for plate in coarse_plates:
        sub = plate.subdivide(depth=subdivisions, min_size=min_subplate_size)
        plates.extend(sub)
    
    # Calculate ocean distances
    grid.calculate_distances_from_ocean(plates)
    
    # Get boundaries for ridge-based terrain
    boundaries = Plate.boundaries(plates)
    
    # Initialize elevations
    elevations = np.zeros(len(grid.hexes))
    
    # Generate elevations based on formation type
    if formation_type == 'ocean_distance':
        elevations = _elevation_ocean_distance(grid, plates, params, elevation_scale)
    elif formation_type == 'ridge':
        elevations = _elevation_ridge(grid, plates, boundaries, params, elevation_scale)
    elif formation_type == 'volcanic':
        elevations = _elevation_volcanic(grid, plates, params, elevation_scale)
    elif formation_type == 'rolling':
        elevations = _elevation_rolling(grid, plates, params, elevation_scale)
    else:
        raise ValueError(f"Unknown formation_type: {formation_type}")
    
    # Apply smoothing based on age
    for _ in range(params['smoothing']):
        elevations = _smooth_elevations(grid, elevations)
    
    terrain.elevations = elevations
    terrain.colorMap()
    
    return terrain, plates


def _elevation_ocean_distance(grid, plates, params, scale):
    """Original approach: elevation based on distance from ocean."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -100 * scale
            var = params['variation'] * 0.5
        else:
            base = (50 + plate.oceanDistance * params['slope']) * scale
            var = params['variation']
        
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_ridge(grid, plates, boundaries, params, scale):
    """Mountains form at plate boundaries."""
    elevations = np.zeros(len(grid.hexes))
    
    # Build plate lookup
    plate_map = {}
    for plate in plates:
        for idx in plate.hexes:
            plate_map[idx] = plate
    
    for i, hx in enumerate(grid.hexes):
        plate = plate_map.get(i)
        if plate is None:
            continue
        
        pos = grid.index_to_hexposition(i)
        
        # Distance to nearest boundary
        if boundaries:
            dist_to_boundary = min(pos.distance(b) for b in boundaries)
        else:
            dist_to_boundary = 10
        
        if plate.kind == PlateKind.oceanic:
            # Ocean: slight trench near boundaries
            base = -80 * scale
            boundary_effect = -max(0, 2 - dist_to_boundary) * 10 * scale
        else:
            # Continental: ridges at boundaries
            base = 30 * scale
            ridge_height = max(0, 4 - dist_to_boundary) * params['peak_height'] / 4 * scale
            boundary_effect = ridge_height
        
        var = params['variation'] * 0.5
        elevations[i] = base + boundary_effect + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_volcanic(grid, plates, params, scale):
    """Peaks at plate centroids (volcanic islands, mountain cores)."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        centroid = plate.centroid_position()
        
        if plate.kind == PlateKind.oceanic:
            base = -60 * scale
            peak = params['peak_height'] * 0.3 * scale  # Underwater volcanoes
        else:
            base = 40 * scale
            peak = params['peak_height'] * scale
        
        for idx in plate.hexes:
            pos = grid.index_to_hexposition(idx)
            dist = pos.distance(centroid)
            
            # Peak at center, falls off
            height = max(0, peak - dist * params['slope'])
            var = params['variation'] * 0.3
            elevations[idx] = base + height + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_rolling(grid, plates, params, scale):
    """Gentle rolling terrain with minimal features."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -30 * scale
        else:
            base = 20 + plate.oceanDistance * params['slope'] * 0.3
            base *= scale
        
        var = params['variation'] * 0.7
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _smooth_elevations(grid, elevations):
    """Single smoothing pass averaging with neighbors."""
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    return smoothed


Terrain.fromSeeds = generate_plate_terrain


In [ ]:
def demoTerr():

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    # Ridge mountains
    t2, p2 = Terrain.fromSeeds(myBounds, radius=20, num_plates=6, oceanic_sides=['N','E'],
                                formation_type='ridge', age='young', seed=42)
    #t2.hexGrid.builder.show()
    t2.colorMap()
    t2.hexGrid.update()

    return t2.hexGrid.builder.show()

In [ ]:
demoTerr()

I didn't see many ridges

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The ridge effect might be too subtle. A few potential issues:

1. **Too many boundaries** - With `subdivisions=3`, there are boundaries everywhere, diluting the effect
2. **Falloff too short** - `max(0, 4 - dist)` only affects hexes within 4 steps
3. **Base elevation dominates** - The ridge height gets lost in the noise

Let's debug and fix:

```python
def demoTerr(formation='ridge', subdivisions=1, age='young'):
    mySize = MapSize(480, 480)
    myBounds = MapRect(MapCord(0,0), mySize)
    
    t, plates = Terrain.fromSeeds(
        myBounds, radius=15, num_plates=5, 
        oceanic_sides=['N','E'],
        formation_type=formation, 
        age=age, 
        subdivisions=subdivisions,  # Fewer subdivisions = clearer boundaries
        seed=42
    )
    t.colorMap()
    t.hexGrid.update()
    
    print(f"Plates: {len(plates)}, Boundaries: {len(Plate.boundaries(plates))}")
    print(f"Elevation range: {min(t.elevations):.0f} to {max(t.elevations):.0f}")
    
    return t.hexGrid.builder.show()

# Try with fewer subdivisions first
demoTerr(subdivisions=1)
```

If ridges are still weak, here's a stronger `_elevation_ridge`:

```python
def _elevation_ridge(grid, plates, boundaries, params, scale):
    """Mountains form at plate boundaries - STRONGER version."""
    elevations = np.zeros(len(grid.hexes))
    
    plate_map = {idx: plate for plate in plates for idx in plate.hexes}
    
    for i, hx in enumerate(grid.hexes):
        plate = plate_map.get(i)
        if plate is None:
            continue
        
        pos = grid.index_to_hexposition(i)
        
        # Distance to nearest boundary
        if boundaries:
            dist_to_boundary = min(pos.distance(b) for b in boundaries)
        else:
            dist_to_boundary = 10
        
        if plate.kind == PlateKind.oceanic:
            base = -80 * scale
            boundary_effect = -max(0, 3 - dist_to_boundary) * 15 * scale
        else:
            # Stronger ridge effect with wider falloff
            base = 20 * scale  # Lower base
            ridge_radius = 6  # Wider ridge influence
            ridge_height = max(0, ridge_radius - dist_to_boundary) * params['peak_height'] / 3 * scale
            boundary_effect = ridge_height
        
        var = params['variation'] * 0.3  # Less noise
        elevations[i] = base + boundary_effect + np.random.uniform(-var, var) * scale
    
    return elevations
```

Key changes:
- **Wider ridge radius** (6 instead of 4)
- **Lower base elevation** so ridges stand out more
- **Less variation** so ridges aren't buried in noise
- **Fewer subdivisions** = fewer, more distinct boundaries

In [ ]:
def demoTerr(formation='ridge', subdivisions=1, age='young'):
    mySize = MapSize(480, 480)
    myBounds = MapRect(MapCord(0,0), mySize)
    
    t, plates = Terrain.fromSeeds(
        myBounds, radius=15, num_plates=5, 
        oceanic_sides=['N','E'],
        formation_type=formation, 
        age=age, 
        subdivisions=subdivisions,  # Fewer subdivisions = clearer boundaries
        seed=42
    )
    t.colorMap()
    t.hexGrid.update()
    
    print(f"Plates: {len(plates)}, Boundaries: {len(Plate.boundaries(plates))}")
    print(f"Elevation range: {min(t.elevations):.0f} to {max(t.elevations):.0f}")
    
    return t

# Try with fewer subdivisions first
sampleTerr = demoTerr(subdivisions=1)
sampleTerr.hexGrid.builder.show()


Plates: 18, Boundaries: 423
Elevation range: -118 to 246


If I wanted to rescale sampleTerr elevations np array so the highest is more of a mountain is there an easy transform that will do it. I know multiply works, is anything better

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

A few options beyond multiply:

```python
# 1. Rescale to target range (preserves shape, changes scale)
def rescale(arr, new_min, new_max):
    old_min, old_max = arr.min(), arr.max()
    return (arr - old_min) / (old_max - old_min) * (new_max - new_min) + new_min

sampleTerr.elevations = rescale(sampleTerr.elevations, -100, 500)

# 2. Power transform (makes peaks more dramatic, flattens lows)
normalized = (sampleTerr.elevations - sampleTerr.elevations.min()) / (sampleTerr.elevations.max() - sampleTerr.elevations.min())
sampleTerr.elevations = normalized ** 1.5  # Try 1.5, 2, etc - higher = sharper peaks

# 3. Only boost the highs (leave ocean alone)
elev = sampleTerr.elevations
land_mask = elev > 0
elev[land_mask] = elev[land_mask] * 2  # Double land heights only

# 4. Exponential curve (most dramatic mountain effect)
normalized = (elev - elev.min()) / (elev.max() - elev.min())
sampleTerr.elevations = np.expm1(normalized * 3)  # e^(3x) - 1
```

For ridges specifically, **power transform** (#2) is probably what you want - it preserves relative positions but makes the high points stand out more relative to the plains.

In [ ]:
normalized = (sampleTerr.elevations - sampleTerr.elevations.min()) / (sampleTerr.elevations.max() - sampleTerr.elevations.min())
sampleTerr.elevations = normalized ** 1.5 

In [ ]:
normalized = (sampleTerr.elevations - sampleTerr.elevations.min()) / (sampleTerr.elevations.max() - sampleTerr.elevations.min())
shaped = normalized ** 1.5  # 0-1 with sharper peaks
sampleTerr.elevations = shaped * 500 - 50  # Rescale to -100 to 300 (or whatever range you want)

sampleTerr.colorMap()
sampleTerr.hexGrid.update()
sampleTerr.hexGrid.builder.show()
